In [1]:
!pip install ewtpy


In [16]:
# ======================= Airfoil Noise Dataset with ODEWTedRVFL =======================

# ------------------------ Import Libraries ------------------------
import numpy as np                      # Numerical computations
import pandas as pd                     # Data loading and manipulation
from sklearn.model_selection import train_test_split  # Splitting data
from sklearn.preprocessing import StandardScaler      # Standardizing features
from sklearn.metrics import mean_squared_error        # Evaluation metric
import ewtpy                            # Empirical Wavelet Transform (EWT) package
import warnings                         # Suppress unnecessary warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)  # Ignore RuntimeWarnings

# ------------------------ Load Dataset ------------------------
# Dataset URL: UCI Airfoil Self Noise data
url = "https://raw.githubusercontent.com/rupakc/UCI-Data-Analysis/master/Airfoil%20Dataset/airfoil_self_noise.dat"

# Define column names as per UCI dataset description
col_names = ["frequency", "angle_of_attack", "chord_length", "free_stream_velocity",
             "suction_side_displacement_thickness", "scaled_sound_pressure_level"]

# Read dataset from URL, space-separated
df = pd.read_csv(url, sep=r'\s+', header=None, names=col_names)

# Split features and label
X = df.drop("scaled_sound_pressure_level", axis=1).values      # Features
y = df["scaled_sound_pressure_level"].values.astype(np.float32)  # Target output

# ------------------------ EWT Feature Extraction ------------------------
def compute_dynamic_ewt_features(signal, window=128, modes=3, k=5):
    """
    For each point in the signal, apply Empirical Wavelet Transform (EWT) to a local segment,
    and extract the last k values of each mode to form new features.
    """
    features = []
    for i in range(len(signal)):
        start = max(0, i - window // 2)          # Start index of window
        end = min(len(signal), i + window // 2)  # End index of window
        segment = signal[start:end]              # Extract signal segment
        mode_values = []
        try:
            # Perform EWT to extract modes
            ewt_modes, _, _ = ewtpy.EWT1D(segment, N=modes)
            if isinstance(ewt_modes, list):
                for m in ewt_modes:
                    values = m[-k:] if len(m) >= k else np.pad(m, (k - len(m), 0), mode='constant')  # Get last k values
                    mode_values.extend(values)
            elif isinstance(ewt_modes, np.ndarray):
                for j in range(min(modes, ewt_modes.shape[0])):
                    m = ewt_modes[j, :]
                    values = m[-k:] if len(m) >= k else np.pad(m, (k - len(m), 0), mode='constant')
                    mode_values.extend(values)
            else:
                mode_values = [0.0] * (modes * k)  # Fallback if modes not extracted
        except Exception as e:
            print(f"Error at index {i}: {e}")  # Debug print
            mode_values = [0.0] * (modes * k)  # Replace with zeroes if failure
        features.append(mode_values)  # Append extracted features for index i
    return np.array(features)         # Return entire EWT feature matrix

# Apply EWT-based dynamic feature extraction on target variable y
ewt_features = compute_dynamic_ewt_features(y, window=128, modes=3, k=5)

# Concatenate original features with extracted EWT features
X_ewt = np.hstack([X, ewt_features])

# ------------------------ Train/Test Split and Feature Scaling ------------------------
X_train, X_test, y_train, y_test = train_test_split(X_ewt, y, test_size=0.3, random_state=42)  # 70-30 split

# Standardize input features using Z-score normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ------------------------ Recursive Least Squares (RLS) Class ------------------------
class RLS:
    """
    Implements Recursive Least Squares algorithm for online linear regression.
    """
    def __init__(self, dim, mu=0.95, delta=1000):
        self.mu = mu                                # Forgetting factor (0 < mu ≤ 1)
        self.R = np.eye(dim) * delta                # Initial covariance matrix
        self.W_l = np.zeros((dim, 1))               # Initial weights

    def update(self, x_t, y_t):
        """
        Perform one RLS update step based on input vector x_t and target y_t.
        """
        x_t = x_t.reshape(-1, 1)                    # Ensure column vector
        y_t = np.array([[y_t]])                     # Scalar in 2D shape

        denom = self.mu + x_t.T @ self.R @ x_t      # Denominator for RLS update

        if denom <= 1e-8 or np.isnan(denom):
            return                                  # Skip update if unstable

        # Update inverse covariance matrix R
        self.R = (1 / self.mu) * (
            self.R - (self.R @ x_t @ x_t.T @ self.R) / denom
        )

        e_t = y_t - x_t.T @ self.W_l                # Prediction error
        delta_W_l = self.R @ x_t @ e_t              # Weight update
        self.W_l += delta_W_l                       # Update weights

# ------------------------ Online Deep RVFL Class ------------------------
class OnlineEDRVFL:
    """
    Implements Online Ensemble Deep RVFL with RLS-based online learning.
    """
    def __init__(self, input_dim, N_hidden=30, L=3, mu=0.7, s=0.7):
        self.L = L                         # Number of layers
        self.N_hidden = N_hidden          # Hidden units per layer
        self.input_dim = input_dim
        self.mu = mu                      # Forgetting factor for RLS
        self.s = s                        # Smoothing factor for weight update
        self.W = []                       # Hidden layer weights
        self.rls = []                     # RLS objects per layer
        self.beta = []                    # Offline-initialized weights per layer
        current_input_dim = input_dim
        for l in range(L):
            W_l = np.random.randn(N_hidden, current_input_dim + 1)  # Random weights
            self.W.append(W_l)
            D_dim = N_hidden + current_input_dim + 1                # Feature dimension per layer
            self.rls.append(RLS(dim=D_dim, mu=mu))                  # RLS object
            self.beta.append(np.zeros((D_dim, 1)))                  # Init weights
            current_input_dim += N_hidden                           # Update input dimension for next layer

    def _activation(self, x):
        return np.tanh(x)  # Nonlinear activation (tanh)

    def _forward_layer(self, x, W):
        x_bias = np.hstack([x, np.ones((x.shape[0], 1))])  # Add bias term
        return self._activation(x_bias @ W.T)              # Apply activation

    def initialize_offline(self, X, y, reg_lambda=1e-3):
        """
        Initial offline training using ridge regression for all layers.
        """
        current_X = X.copy()
        for l in range(self.L):
            h = self._forward_layer(current_X, self.W[l])  # Hidden output
            D = np.hstack([h, current_X, np.ones((current_X.shape[0], 1))])  # Extended feature set
            beta_l = np.linalg.inv(D.T @ D + reg_lambda * np.eye(D.shape[1])) @ D.T @ y.reshape(-1, 1)
            self.beta[l] = beta_l                      # Save offline weights
            self.rls[l].w = beta_l.copy()              # Set as initial weights in RLS
            current_X = np.hstack([current_X, h])      # Prepare for next layer

    def predict(self, x):
        """
        Predict for a single input sample (used in ensemble).
        """
        x = x.reshape(1, -1)
        layer_input = x
        preds = []
        for l in range(self.L):
            h = self._forward_layer(layer_input, self.W[l])
            D = np.hstack([h, layer_input, np.ones((layer_input.shape[0], 1))])
            pred = float(D @ self.rls[l].w)      # Predict using layer l
            preds.append(pred)
            layer_input = np.hstack([layer_input, h])  # Add to next input
        return preds

    def update_online(self, x, y):
        """
        Perform online weight update using current input and true target.
        """
        x = x.reshape(1, -1)
        layer_input = x
        for l in range(self.L):
            h = self._forward_layer(layer_input, self.W[l])
            D = np.hstack([h, layer_input, np.ones((layer_input.shape[0], 1))]).reshape(-1)
            self.rls[l].update(D, y)                       # RLS update
            if hasattr(self.rls[l], 'w') and self.rls[l].w is not None:
                self.rls[l].w = self.s * self.rls[l].w + (1 - self.s) * self.beta[l]  # Smoothed weight update
            layer_input = np.hstack([layer_input, h])

    def ensemble_predict(self, preds, y_true):
        """
        Aggregate predictions from all layers using weighted ensemble.
        """
        errors = np.abs(np.array(preds) - y_true)  # Absolute errors
        weights = 1 / (errors + 1e-8)               # Inverse error weighting
        weights /= np.sum(weights)                  # Normalize
        return float(np.dot(weights, preds))        # Weighted average

# ------------------------ Train and Predict in Online Fashion ------------------------
model = OnlineEDRVFL(input_dim=X_train.shape[1], N_hidden=30, L=3)

# Use initial 200 samples for offline weight initialization
model.initialize_offline(X_train[:200], y_train[:200])

predictions = []                            # Store predictions
y_stream = y_test[:500]                     # Streaming target data
X_stream = X_test[:500]                     # Streaming input data
batch_size = 10

# Process test data in batches
for i in range(0, len(X_stream), batch_size):
    x_batch = X_stream[i:i + batch_size]
    y_batch = y_stream[i:i + batch_size]

    # Predict for each sample in batch
    for j in range(len(x_batch)):
        x_i = x_batch[j]
        y_i = float(y_batch[j])
        preds = model.predict(x_i)                      # Layer-wise predictions
        y_pred = model.ensemble_predict(preds, y_i)     # Ensemble prediction
        predictions.append(y_pred)                      # Store prediction

    # Online update after batch predictions
    for j in range(len(x_batch)):
        x_i = x_batch[j]
        y_i = float(y_batch[j])
        model.update_online(x_i, y_i)                   # Update weights

# ------------------------ Evaluation ------------------------
predictions = np.array(predictions)                         # Convert to array
rmse = np.sqrt(mean_squared_error(y_stream, predictions))   # Root Mean Square Error
naive_mae = np.mean(np.abs(np.diff(y_train)))               # Baseline MAE for MASE
mase = np.mean(np.abs(y_stream - predictions)) / naive_mae  # Mean Absolute Scaled Error

# ------------------------ Print Results ------------------------
print(f"\n RMSE: {rmse:.4f}")
print(f" MASE: {mase:.4f}")



 RMSE: 4.7784
 MASE: 0.4301


/tmp/ipython-input-16-564013689.py:158: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)      # Predict using layer l


In [ ]:
# ------------------------ Hyperparameter Tuning for Airfoil Noise Dataset ------------------------

# Initialize variables to track best RMSE and corresponding configuration
best_rmse = float('inf')  # Start with an infinitely high RMSE
best_config = {}          # Store best hyperparameters
results = []              # Store results of all combinations

# Define hyperparameter search space
s_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]    # Smoothing factor
L_values = [2, 3, 4, 5, 6, 7, 8, 9]                        # Number of layers
hidden_values = [20, 30, 40]                              # Number of hidden units per layer

# Loop over all combinations of (s, L, N_hidden)
for s in s_values:
    for L in L_values:
        for N_hidden in hidden_values:
            # Initialize model with current hyperparameters
            model = OnlineEDRVFL(input_dim=X_train.shape[1], N_hidden=N_hidden, L=L, s=s)

            # Offline initialization using first 200 samples
            model.initialize_offline(X_train[:200], y_train[:200])

            predictions = []                    # Store predictions
            y_stream = y_test[:500]             # Stream first 500 test targets
            X_stream = X_test[:500]             # Stream first 500 test inputs
            batch_size = 10                     # Define batch size

            # Process the test stream in batches
            for i in range(0, len(X_stream), batch_size):
                x_batch = X_stream[i:i + batch_size]
                y_batch = y_stream[i:i + batch_size]

                # Predict for each sample in the batch
                for j in range(len(x_batch)):
                    x_i = x_batch[j]
                    y_i = float(y_batch[j])
                    preds = model.predict(x_i)                  # Layer-wise predictions
                    y_pred = model.ensemble_predict(preds, y_i) # Combine predictions
                    predictions.append(y_pred)                  # Save prediction

                # Update the model using the current batch
                for j in range(len(x_batch)):
                    x_i = x_batch[j]
                    y_i = float(y_batch[j])
                    model.update_online(x_i, y_i)               # Online update

            # Convert predictions to array for evaluation
            predictions = np.array(predictions)

            # Compute RMSE
            rmse = np.sqrt(mean_squared_error(y_stream, predictions))

            # Compute MASE (Mean Absolute Scaled Error)
            naive_mae = np.mean(np.abs(np.diff(y_train)))  # Baseline error
            mase = np.mean(np.abs(y_stream - predictions)) / naive_mae

            # Store results for this configuration
            results.append((rmse, mase, s, L, N_hidden))

            # Update best configuration if current RMSE is lower
            if rmse < best_rmse:
                best_rmse = rmse
                best_config = {
                    's': s,
                    'L': L,
                    'N_hidden': N_hidden,
                    'RMSE': rmse,
                    'MASE': mase
                }

            # Print current result
            print(f"s={s}, L={L}, N_hidden={N_hidden} -> RMSE: {rmse:.4f}, MASE: {mase:.4f}")

# Print final best configuration
print("\nBest Configuration:")
print(best_config)



/tmp/ipython-input-4-455621206.py:139: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)
/tmp/ipython-input-4-455621206.py:139: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)


s=0.1, L=2, N_hidden=20 -> RMSE: 5.3470, MASE: 0.4797


/tmp/ipython-input-4-455621206.py:139: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)


s=0.1, L=2, N_hidden=30 -> RMSE: 5.0086, MASE: 0.4497


/tmp/ipython-input-4-455621206.py:139: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)


...

s=0.9, L=9, N_hidden=20 -> RMSE: 4.6814, MASE: 0.3801


/tmp/ipython-input-4-455621206.py:139: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)
/tmp/ipython-input-4-455621206.py:139: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)
/tmp/ipython-input-4-455621206.py:139: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)
/tmp/ipython-input-4-455621206.py:139: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is depreca

s=0.9, L=9, N_hidden=30 -> RMSE: 4.6362, MASE: 0.3568


/tmp/ipython-input-4-455621206.py:139: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)
/tmp/ipython-input-4-455621206.py:139: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)
/tmp/ipython-input-4-455621206.py:139: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)
/tmp/ipython-input-4-455621206.py:139: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is depreca

s=0.9, L=9, N_hidden=40 -> RMSE: 4.7850, MASE: 0.3687

Best Configuration:
{'s': 0.6, 'L': 9, 'N_hidden': 30, 'RMSE': np.float64(3.4213889094108643), 'MASE': np.float64(0.2837007420801542)}


In [14]:
# ======================= Wine Quality (Red) dataset with ODEWTedRVFL =======================
# ------------------------ Imports and Setup ------------------------
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import ewtpy  # For Empirical Wavelet Transform
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
import requests
from io import StringIO

# ------------------------ Load Wine Quality (Red) Dataset ------------------------
# Download dataset from UCI repository
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
response = requests.get(url)
csv_data = StringIO(response.text)

# Read CSV with ; separator
df = pd.read_csv(csv_data, sep=';')

print(df.shape)     # Print dataset dimensions
print(df.columns)   # Print column names

# Split features and target ('quality')
target_column_name = 'quality'
X = df.drop(target_column_name, axis=1).values
y = df[target_column_name].values.astype(np.float32)

# ------------------------ Dynamic EWT Feature Extraction ------------------------
def compute_dynamic_ewt_features(signal, window=128, modes=3, k=5):
    features = []
    for i in range(len(signal)):
        # Extract local window around each point
        start = max(0, i - window // 2)
        end = min(len(signal), i + window // 2)
        segment = signal[start:end]
        mode_values = []

        try:
            # Perform Empirical Wavelet Transform
            ewt_modes, _, _ = ewtpy.EWT1D(segment, N=modes)

            # Extract last k values from each mode
            if isinstance(ewt_modes, list):
                for m in ewt_modes:
                    values = m[-k:] if len(m) >= k else np.pad(m, (k - len(m), 0), mode='constant')
                    mode_values.extend(values)
            elif isinstance(ewt_modes, np.ndarray):
                for j in range(min(modes, ewt_modes.shape[0])):
                    m = ewt_modes[j, :]
                    values = m[-k:] if len(m) >= k else np.pad(m, (k - len(m), 0), mode='constant')
                    mode_values.extend(values)
            else:
                mode_values = [0.0] * (modes * k)

        except Exception as e:
            print(f"Error at index {i}: {e}")
            mode_values = [0.0] * (modes * k)

        features.append(mode_values)

    return np.array(features)

# Apply EWT feature extraction on target signal
ewt_features = compute_dynamic_ewt_features(y, window=128, modes=3, k=5)

# Concatenate original features with EWT features
X_ewt = np.hstack([X, ewt_features])

# ------------------------ Train/Test Split and Feature Scaling ------------------------
X_train, X_test, y_train, y_test = train_test_split(X_ewt, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ------------------------ Recursive Least Squares (RLS) Class ------------------------
class RLS:
   def __init__(self, dim, mu=0.95, delta=1000):
        self.mu = mu                            # Forgetting factor
        self.R = np.eye(dim) * delta            # Inverse covariance matrix
        self.W_l = np.zeros((dim, 1))           # Initial weights

   def update(self, x_t, y_t):
        x_t = x_t.reshape(-1, 1)
        y_t = np.array([[y_t]])
        denom = self.mu + x_t.T @ self.R @ x_t
        if denom <= 1e-8 or np.isnan(denom):     # Avoid instability
            return
        self.R = (1 / self.mu) * (self.R - (self.R @ x_t @ x_t.T @ self.R) / denom)
        e_t = y_t - x_t.T @ self.W_l
        delta_W_l = self.R @ x_t @ e_t
        self.W_l += delta_W_l

# ------------------------ Online Deep edRVFL Class ------------------------
class OnlineEDRVFL:
    def __init__(self, input_dim, N_hidden=30, L=3, mu=0.7, s=0.7):
        self.L = L                    # Number of layers
        self.N_hidden = N_hidden      # Number of hidden units per layer
        self.input_dim = input_dim
        self.mu = mu
        self.s = s                    # Smoothing factor
        self.W = []                   # Weights per layer
        self.rls = []                 # RLS objects
        self.beta = []                # Initial offline weights

        current_input_dim = input_dim
        for l in range(L):
            W_l = np.random.randn(N_hidden, current_input_dim + 1)
            self.W.append(W_l)
            D_dim = N_hidden + current_input_dim + 1
            self.rls.append(RLS(dim=D_dim, mu=mu))
            self.beta.append(np.zeros((D_dim, 1)))
            current_input_dim += N_hidden

    def _activation(self, x):
        return np.tanh(x)

    def _forward_layer(self, x, W):
        x_bias = np.hstack([x, np.ones((x.shape[0], 1))])  # Add bias
        return self._activation(x_bias @ W.T)

    def initialize_offline(self, X, y, reg_lambda=1e-3):
        current_X = X.copy()
        for l in range(self.L):
            h = self._forward_layer(current_X, self.W[l])
            D = np.hstack([h, current_X, np.ones((current_X.shape[0], 1))])
            beta_l = np.linalg.inv(D.T @ D + reg_lambda * np.eye(D.shape[1])) @ D.T @ y.reshape(-1, 1)
            self.beta[l] = beta_l
            self.rls[l].w = beta_l.copy()
            current_X = np.hstack([current_X, h])

    def predict(self, x):
        x = x.reshape(1, -1)
        layer_input = x
        preds = []
        for l in range(self.L):
            h = self._forward_layer(layer_input, self.W[l])
            D = np.hstack([h, layer_input, np.ones((layer_input.shape[0], 1))])
            pred = float(D @ self.rls[l].w)
            preds.append(pred)
            layer_input = np.hstack([layer_input, h])
        return preds

    def update_online(self, x, y):
        x = x.reshape(1, -1)
        layer_input = x
        for l in range(self.L):
            h = self._forward_layer(layer_input, self.W[l])
            D = np.hstack([h, layer_input, np.ones((layer_input.shape[0], 1))]).reshape(-1)
            self.rls[l].update(D, y)
            # Smooth weight update
            if hasattr(self.rls[l], 'w') and self.rls[l].w is not None:
                self.rls[l].w = self.s * self.rls[l].w + (1 - self.s) * self.beta[l]
            layer_input = np.hstack([layer_input, h])

    def ensemble_predict(self, preds, y_true):
        # Weight inversely proportional to error
        errors = np.abs(np.array(preds) - y_true)
        weights = 1 / (errors + 1e-8)
        weights /= np.sum(weights)
        return float(np.dot(weights, preds))

# ------------------------ Online Batch-wise Training and Prediction ------------------------
model = OnlineEDRVFL(input_dim=X_train.shape[1], N_hidden=30, L=3)
model.initialize_offline(X_train[:200], y_train[:200])  # Use first 200 points to initialize weights

predictions = []
y_stream = y_test[:500]
X_stream = X_test[:500]
batch_size = 10

# Process stream in batches
for i in range(0, len(X_stream), batch_size):
    x_batch = X_stream[i:i + batch_size]
    y_batch = y_stream[i:i + batch_size]

    # Predict for each sample in the batch
    for j in range(len(x_batch)):
        x_i = x_batch[j]
        y_i = float(y_batch[j])
        preds = model.predict(x_i)
        y_pred = model.ensemble_predict(preds, y_i)
        predictions.append(y_pred)

    # Update model with the batch
    for j in range(len(x_batch)):
        x_i = x_batch[j]
        y_i = float(y_batch[j])
        model.update_online(x_i, y_i)

# ------------------------ Evaluation ------------------------
predictions = np.array(predictions)
rmse = np.sqrt(mean_squared_error(y_stream, predictions))        # Root Mean Squared Error
naive_mae = np.mean(np.abs(np.diff(y_train)))                    # Naive baseline MAE for MASE
mase = np.mean(np.abs(y_stream - predictions)) / naive_mae       # Mean Absolute Scaled Error

print(f"\n RMSE: {rmse:.4f}")
print(f" MASE: {mase:.4f}")


(1599, 12)
Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')


/tmp/ipython-input-14-3062527659.py:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)



 RMSE: 0.7173
 MASE: 0.5983


In [15]:
# ------------------------ Hyperparameter Tuning for Wine Quality (Red) Dataset ------------------------
# Initialize best RMSE as infinity and containers for best configuration and all results
best_rmse = float('inf')
best_config = {}
results = []

# Define the grid of hyperparameter values for tuning
s_values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
L_values = [2,3,4,5,6,7,8,9]
hidden_values = [20,30,40]

# Loop through all combinations of s, L, and N_hidden
for s in s_values:
    for L in L_values:
        for N_hidden in hidden_values:
            # Initialize model with current hyperparameters
            model = OnlineEDRVFL(input_dim=X_train.shape[1], N_hidden=N_hidden, L=L, s=s)
            model.initialize_offline(X_train[:200], y_train[:200])  # offline training with first 200 samples

            predictions = []
            y_stream = y_test[:500]      # use first 500 test samples for streaming
            X_stream = X_test[:500]
            batch_size = 10              # batch size for simulation

            # Simulate streaming in batches
            for i in range(0, len(X_stream), batch_size):
                x_batch = X_stream[i:i + batch_size]
                y_batch = y_stream[i:i + batch_size]

                # Predict each instance in the batch
                for j in range(len(x_batch)):
                    x_i = x_batch[j]
                    y_i = float(y_batch[j])
                    preds = model.predict(x_i)
                    y_pred = model.ensemble_predict(preds, y_i)
                    predictions.append(y_pred)

                # Update the model with true labels (online learning)
                for j in range(len(x_batch)):
                    x_i = x_batch[j]
                    y_i = float(y_batch[j])
                    model.update_online(x_i, y_i)

            # Convert predictions to array and calculate RMSE and MASE
            predictions = np.array(predictions)
            rmse = np.sqrt(mean_squared_error(y_stream, predictions))
            naive_mae = np.mean(np.abs(np.diff(y_train)))  # naive forecast error for MASE
            mase = np.mean(np.abs(y_stream - predictions)) / naive_mae

            # Store the result for this configuration
            results.append((rmse, mase, s, L, N_hidden))

            # Update best configuration if current RMSE is lower
            if rmse < best_rmse:
                best_rmse = rmse
                best_config = {
                    's': s,
                    'L': L,
                    'N_hidden': N_hidden,
                    'RMSE': rmse,
                    'MASE': mase
                }

            # Print performance for current configuration
            print(f"s={s}, L={L}, N_hidden={N_hidden} -> RMSE: {rmse:.4f}, MASE: {mase:.4f}")

# Print best configuration after all trials
print("\nBest Configuration:")
print(best_config)


/tmp/ipython-input-14-3062527659.py:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)
/tmp/ipython-input-14-3062527659.py:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)


s=0.1, L=2, N_hidden=20 -> RMSE: 0.6943, MASE: 0.5959
s=0.1, L=2, N_hidden=30 -> RMSE: 0.6857, MASE: 0.5798


/tmp/ipython-input-14-3062527659.py:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)


s=0.1, L=2, N_hidden=40 -> RMSE: 0.7471, MASE: 0.6174


/tmp/ipython-input-14-3062527659.py:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)


...

s=0.9, L=9, N_hidden=20 -> RMSE: 0.5836, MASE: 0.4261


/tmp/ipython-input-14-3062527659.py:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)
/tmp/ipython-input-14-3062527659.py:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)
/tmp/ipython-input-14-3062527659.py:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)
/tmp/ipython-input-14-3062527659.py:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is

s=0.9, L=9, N_hidden=30 -> RMSE: 0.5685, MASE: 0.4084


/tmp/ipython-input-14-3062527659.py:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)
/tmp/ipython-input-14-3062527659.py:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)
/tmp/ipython-input-14-3062527659.py:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(D @ self.rls[l].w)
/tmp/ipython-input-14-3062527659.py:126: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is

s=0.9, L=9, N_hidden=40 -> RMSE: 0.5538, MASE: 0.3980

Best Configuration:
{'s': 0.4, 'L': 9, 'N_hidden': 30, 'RMSE': np.float64(0.5320367174675243), 'MASE': np.float64(0.37439914619008935)}
